# Pre-pro

In [232]:
!pip install wandb
!pip install transformers
!pip install sentence_transformers
!pip install pipeline

In [233]:
import pandas as pd
import tensorflow_hub as hub
import numpy as np
import torch
from torch.nn.functional import softmax
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import LongformerTokenizer, LongformerForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification

from math import *

import sys
sys.path.append('models')
import matplotlib.pyplot as plt
%matplotlib inline
import wandb
from wandb.keras import WandbCallback
import json
from google.colab import drive
drive.mount('/content/gdrive')
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, get_linear_schedule_with_warmup

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [234]:
from sklearn.metrics import confusion_matrix
import plotly.express as px
def show_conf_matrix(y_test, pred, labels_name):
  list_cat= list(set(y_test))
  conf_matrix = confusion_matrix(y_test, pred, labels=list_cat)
  cf = [[i/sum(ligne) for i in ligne] for ligne in conf_matrix]
  data = cf
  fig = px.imshow(data,
                  labels={'y' : "Vrai Catégorie", 'x' :"Catégorie prédite", 'color' : "Productivity"},
                  x=labels_name,
                  y=labels_name
                )
  fig.update_xaxes(side="top")
  fig.show()

In [235]:
def function_reu(x):
  try:
    return dic[x]
  except:
    return ' '
def function_test(x):
  try:
    return dictionnary[(x+ '.mp3')]
  except:
    return 'do not have'

In [236]:
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans0.txt', encoding='utf-16')
text0 = file.read()
d = json.loads(text0)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans1.txt', encoding='utf-16')
text1 = file.read()
d1 = json.loads(text1)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans2.txt', encoding='utf-16')
text2 = file.read()
d2 = json.loads(text2)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans_rest.txt', encoding='utf-16')
text3 = file.read()
d3 = json.loads(text3)
dictionnary = d | d1
dictionnary = dictionnary | d2
dictionnary = dictionnary | d3
len(dictionnary)
descr = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/Transcription_wav.csv')
name_video = descr['name_vid']
contenue = descr['contenue']
dic = {}
vid_name_set = set()
for vid_name, cont in zip(list(name_video), list(contenue)):
  if not 'artificial intelligence' in cont.lower():
    if not vid_name in dic.keys():
      dic[vid_name] = cont
    else:
      dic[vid_name] += cont
  else:
    vid_name_set.add(vid_name)
    print(vid_name)
descr['descr'] = descr['name_vid'].apply(lambda x : function_reu(x))
descr = descr.drop_duplicates(subset='name_vid')
dt = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVT.csv')
dt['name'] = dt['video'].apply(lambda x : x[5:-4])
dt['transcription'] = dt['name'].apply(lambda x : function_test(x))
dt = dt.drop('video', axis=1)
dt = dt[dt.transcription != 'do not have']
intersection = pd.merge(dt, descr, left_on='name', right_on='name_vid')
intersection = intersection.drop('label', axis = 1)
intersection = intersection.drop('Unnamed: 0', axis = 1)
intersection = intersection.drop('name_audio', axis = 1)
intersection = intersection.drop('id_audio', axis = 1)
intersection = intersection.drop('name', axis = 1)
intersection = intersection.drop('name_vid', axis = 1)
intersection = intersection.drop('contenue', axis = 1)
intersection['total'] = intersection['text'] + intersection['descr'] + intersection['transcription']
print(len(intersection), len(dt), len(descr))

-eOrNdBpGMv8
1551 2085 1878


In [237]:
import pickle as pk
with open('/content/gdrive/MyDrive/Projet_Multimedia/download/raw_captions_combined_filtered.pkl', "rb") as fp:
    dict_caption= pk.load(fp)
def liste_strings_concat(x):
  strings = ''
  for elem in x:
    for world in elem:
      strings += world + ' '
  return strings
liste_strings_concat(dict_caption['video-__5k7e0f3r4'])
df = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVD.csv')
df['text'] = df['name'].apply(lambda x : liste_strings_concat(dict_caption['video' + x ]))

In [238]:
df_test = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Test_SurSampling1.csv")
df_train = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Train_SurSampling1.csv")
df_valid = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Val_SurSampling1.csv")
len(df_test), len(df_train), len(df_valid)

(640, 1909, 639)

In [239]:
df_train['text'] = df_train['name'].apply(lambda x : liste_strings_concat(dict_caption['video' + x ]))
df_test['text'] = df_test['name'].apply(lambda x : liste_strings_concat(dict_caption['video' + x ]))
df_valid['text'] = df_valid['name'].apply(lambda x : liste_strings_concat(dict_caption['video' + x ]))

In [240]:
df['label'].value_counts()

Education                732
People & Blogs           340
Entertainment            301
Film & Animation         286
Howto & Style            137
Comedy                    95
Nonprofits & Activism     95
Science & Technology      65
Pets & Animals            53
News & Politics           52
Music                     39
Travel & Events           32
Sports                    21
Gaming                    17
Autos & Vehicles          15
Name: label, dtype: int64

In [241]:
df

,name,label,duree,text
0,-KHQk1_Vq69E,Autos & Vehicles,21,in this video you will learn about a bar graph...
1,-mG4Y2Snygfk,People & Blogs,60,when the screen comes on we see a notebook at ...
2,-OdEfx8up7wM,People & Blogs,69,Jack in the dog looking child grimaces while h...
3,-Opbfh4wNu7Q,Entertainment,61,Captain America Ironman and Thor walk towards ...
4,-A5340xbbQJU,Sports,299,the Swing Shot is fired firing the basketball ...
...,...,...,...,...
2275,-ymc30meWzfg,Education,79,Professor takes the blood of a patient Tiffany...
2276,-h7HGWRTGqyg,Education,164,a blindfold adult wearing green sits next to a...
2277,-XVRLTfN1WYw,Howto & Style,90,add some whipped cream caramel waffle cone a m...
2278,-jTtH7Fq1trw,Travel & Events,185,located in the Sugar Factory Fashion Show in L...


# Deep Learning

## DistilBERT

In [242]:
# Convertir la colonne 'Label' en type 'category'
df['label'] = df['label'].astype('category')

# Séparer les données en ensemble d'entraînement et de test
train_df, test_df = train_test_split(df, test_size=0.15, random_state=2)
train_df, valid_df = train_test_split(train_df, test_size=0.10, random_state=2)
## Important : toujours prendre random_state = 2 pour comparaison!

In [243]:
len(train_df), len(valid_df), len(test_df)

(1744, 194, 342)

In [244]:
train_df['label'].value_counts(), test_df['label'].value_counts()

(Education                553
 People & Blogs           252
 Entertainment            232
 Film & Animation         223
 Howto & Style            114
 Nonprofits & Activism     74
 Comedy                    72
 Science & Technology      52
 Pets & Animals            43
 News & Politics           38
 Music                     29
 Travel & Events           21
 Sports                    16
 Gaming                    14
 Autos & Vehicles          11
 Name: label, dtype: int64,
 Education                109
 People & Blogs            60
 Entertainment             45
 Film & Animation          38
 Comedy                    16
 Nonprofits & Activism     14
 Howto & Style             12
 News & Politics            9
 Pets & Animals             9
 Science & Technology       8
 Travel & Events            8
 Music                      5
 Sports                     4
 Gaming                     3
 Autos & Vehicles           2
 Name: label, dtype: int64)

In [245]:
train_df, test_df, valid_df = df_train, df_test, df_valid
train_df['label'] = df_train['label'].astype('category')
test_df['label'] = df_test['label'].astype('category')
valid_df['label'] = df_valid['label'].astype('category')

In [246]:
len(train_df), len(valid_df), len(test_df)

(1909, 639, 640)

In [247]:
# Charger le tokenizer et le modèle pré-entraîné DistilBERT plus petit
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(df['label'].cat.categories)).to('cuda')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [248]:
label_name = 'text' #what we changed when tried other types of text -> intersection of text (concatenation)

In [249]:
# Tokeniser les textes
train_tokens = tokenizer.batch_encode_plus(train_df[label_name].tolist(), padding=True, truncation=True, return_tensors='pt')
valid_tokens = tokenizer.batch_encode_plus(valid_df[label_name].tolist(), padding=True, truncation=True, return_tensors='pt')
test_tokens = tokenizer.batch_encode_plus(test_df[label_name].tolist(), padding=True, truncation=True, return_tensors='pt')

In [19]:
# Créer des jeux de données PyTorch
train_dataset = TensorDataset(train_tokens['input_ids'].to('cuda'),train_tokens['attention_mask'].to('cuda'),torch.tensor(train_df['label'].cat.codes.tolist(), dtype=torch.long).to('cuda'))
valid_dataset = TensorDataset(valid_tokens['input_ids'].to('cuda'),valid_tokens['attention_mask'].to('cuda'),torch.tensor(valid_df['label'].cat.codes.tolist(), dtype=torch.long).to('cuda'))
test_dataset = TensorDataset(test_tokens['input_ids'].to('cuda'), test_tokens['attention_mask'].to('cuda'), torch.tensor(test_df['label'].cat.codes.tolist(), dtype=torch.long).to('cuda'))

In [ ]:
# Définir le DataLoader
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss_train, total_loss_valid = 0.0, 0.0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_train += loss.item()
        loss.backward()
        optimizer.step()

    for batch in valid_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_valid += loss.item()

    average_loss_train, average_loss_valid = total_loss_train / len(train_loader), total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, loss: {average_loss_train}, val_loss: {average_loss_valid}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



Epoch 1/5, loss: 2.255866608859106, val_loss: 1.9721949219703674
Epoch 2/5, loss: 1.4332818648306396, val_loss: 1.8275840796530247
Epoch 3/5, loss: 0.8163646644129414, val_loss: 1.845581129938364
Epoch 4/5, loss: 0.46159358827638824, val_loss: 2.0316098921000956
Epoch 5/5, loss: 0.24701171907670327, val_loss: 2.2013230569660664


### Resultat label normaux

In [ ]:
# Évaluer le modèle
model.eval()
all_preds = []
all_preds_proba = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_preds_proba.extend(logits)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_sur_gpu.pth')

Accuracy: 0.6052631578947368


In [ ]:
dic = {}
for count, elem in enumerate(torch.tensor(test_df['label'].cat.codes.tolist())):
  if not (elem in dic.keys()):
    dic[elem.item()] = list(test_df['label'])[count]
dic

{2: 'Education',
 10: 'People & Blogs',
 4: 'Film & Animation',
 3: 'Entertainment',
 11: 'Pets & Animals',
 8: 'News & Politics',
 12: 'Science & Technology',
 6: 'Howto & Style',
 9: 'Nonprofits & Activism',
 13: 'Sports',
 14: 'Travel & Events',
 0: 'Autos & Vehicles',
 7: 'Music',
 1: 'Comedy',
 5: 'Gaming'}

In [ ]:
list_cat = []
for i in np.sort(list(set(all_preds))):
  list_cat.append(dic[i])

In [ ]:
print(len(set(all_preds)), len(set(test_df['label'])))
show_conf_matrix(all_preds, torch.tensor(test_df['label'].cat.codes.tolist(), dtype=torch.long), list_cat)

13 15


In [ ]:
for elem in range (15):
  if elem not in all_preds:
    print('there was no prediction of', dic[elem], 'there was ', len(train_df[train_df.label == dic[elem]]), 'elements in the train, and it was supposed to find ', len(test_df[test_df.label == dic[elem]]))

there was no prediction of Autos & Vehicles there was  11 elements in the train, and it was supposed to find  2
there was no prediction of Gaming there was  14 elements in the train, and it was supposed to find  3


### resultat label avec Sur/Sub sampling

In [ ]:
# Évaluer le modèle
model.eval()
all_preds = []
all_preds_proba = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_preds_proba.extend(logits)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_sur_gpu.pth')

Accuracy: 0.4734375


(best result was 47.7%)

In [ ]:
dic = {}
for count, elem in enumerate(torch.tensor(test_df['label'].cat.codes.tolist())):
  if not (elem in dic.keys()):
    dic[elem.item()] = list(test_df['label'])[count]
dic

{4: 'Film & Animation',
 2: 'Education',
 10: 'People & Blogs',
 6: 'Howto & Style',
 13: 'Sports',
 3: 'Entertainment',
 0: 'Autos & Vehicles',
 9: 'Nonprofits & Activism',
 14: 'Travel & Events',
 12: 'Science & Technology',
 5: 'Gaming',
 11: 'Pets & Animals',
 1: 'Comedy',
 8: 'News & Politics',
 7: 'Music'}

In [ ]:
list_cat = []
for i in np.sort(list(set(all_preds))):
  list_cat.append(dic[i])

In [ ]:
print(len(set(all_preds)), len(set(test_df['label'])))
show_conf_matrix(all_preds, torch.tensor(test_df['label'].cat.codes.tolist(), dtype=torch.long), list_cat)

14 15


In [ ]:
for elem in range (15):
  if elem not in all_preds:
    print('there was no prediction of', dic[elem], 'there was ', len(train_df[train_df.label == dic[elem]]), 'elements in the train, and it was supposed to find ', len(test_df[test_df.label == dic[elem]]))

there was no prediction of Autos & Vehicles there was  83 elements in the train, and it was supposed to find  28


In [ ]:
reverse_dict = {v: k for k, v in dic.items()}

In [ ]:
index = np.where(test_df['label'] == 'Autos & Vehicles')[0]
for elem in set(np.array(all_preds)[index]):
  print(dic[elem])

Entertainment


Every autos & vehicles went into Entertainement, when we looked at the video this makes sense.

### Model DP manuel sur sub/sur

In [28]:
# Définir le modèle de classification
class ClassificationModel(torch.nn.Module):
    def __init__(self, num_labels):
        super(ClassificationModel, self).__init__()
        self.fc = torch.nn.Linear(1024, 2048) ##Only last dimension of output is necessary, bach*2,512 -> 2024
        self.r = torch.nn.ReLU()
        self.s = torch.nn.Sigmoid()
        self.d = torch.nn.Dropout(p=0.25)
        self.fc1 = torch.nn.Linear(2048, 2048)
        self.fc2 = torch.nn.Linear(2048, 2048)
        self.fc3 = torch.nn.Linear(2048, 512)
        self.fc4 = torch.nn.Linear(512, num_labels)

    def forward(self, x):
      x = self.d(self.r(self.fc(x)))
      x = self.d(self.r(self.fc1(x)))
      x = self.d(self.r(self.fc2(x)))
      x = self.d(self.r(self.fc3(x)))
      x = self.s(self.fc4(x))
      return x

# Initialiser le modèle de classification
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = ClassificationModel(num_labels = 15).to('cuda')

In [21]:
# Créer des jeux de données PyTorch
train_dataset = TensorDataset(train_tokens['input_ids'].to('cuda'),train_tokens['attention_mask'].to('cuda'),torch.tensor(np.array(pd.get_dummies(train_df['label'])), dtype=torch.float32).to('cuda'))
valid_dataset = TensorDataset(valid_tokens['input_ids'].to('cuda'),valid_tokens['attention_mask'].to('cuda'),torch.tensor(np.array(pd.get_dummies(valid_df['label'])), dtype=torch.float32).to('cuda'))
test_dataset = TensorDataset(test_tokens['input_ids'].to('cuda'), test_tokens['attention_mask'].to('cuda'), torch.tensor(np.array(pd.get_dummies(test_df['label'])), dtype=torch.float32).to('cuda'))

In [23]:
train_tokens['input_ids'].shape,train_tokens['attention_mask'].shape

(torch.Size([1909, 512]), torch.Size([1909, 512]))

In [29]:
# Définir le DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 40

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss_train, total_loss_valid = 0.0, 0.0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(torch.concat((torch.tensor(input_ids, dtype=torch.float32), (torch.tensor(attention_mask, dtype=torch.float32))), 1))
        loss = torch.nn.functional.binary_cross_entropy(outputs, labels)
        total_loss_train += loss.item()
        loss.backward()
        optimizer.step()

    for batch in valid_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(torch.concat((torch.tensor(input_ids, dtype=torch.float32), (torch.tensor(attention_mask, dtype=torch.float32))), 1))
        loss = torch.nn.functional.binary_cross_entropy(outputs, labels)
        total_loss_valid += loss.item()

    average_loss_train, average_loss_valid = total_loss_train / len(train_loader), total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, loss: {average_loss_train}, val_loss: {average_loss_valid}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-29-8d5d239d1cc6>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.concat((torch.tensor(input_ids, dtype=torch.float32), (torch.tensor(attention_mask, dtype=torch.float32))), 1))
<ipython-input-29-8d5d239d1cc6>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.concat((torch.tensor(input_ids, dtype=torch.float32), (torch.tensor

Epoch 1/40, loss: 9.430559086799622, val_loss: 6.113217371702194
Epoch 2/40, loss: 6.085230843226115, val_loss: 6.221832835674286
Epoch 3/40, loss: 6.249782145023346, val_loss: 6.199558484554291
Epoch 4/40, loss: 6.142179807027181, val_loss: 6.4868984580039974
Epoch 5/40, loss: 6.139323226610819, val_loss: 6.221142566204071
Epoch 6/40, loss: 6.18940711816152, val_loss: 6.255258023738861
Epoch 7/40, loss: 6.0781308054924015, val_loss: 6.351089084148407
Epoch 8/40, loss: 5.993546656767527, val_loss: 6.079045343399048
Epoch 9/40, loss: 5.962260719140371, val_loss: 6.190892291069031
Epoch 10/40, loss: 5.886498419443766, val_loss: 6.0427149653434755
Epoch 11/40, loss: 5.9194457054138185, val_loss: 6.365668416023254
Epoch 12/40, loss: 5.793376390139262, val_loss: 6.046955895423889
Epoch 13/40, loss: 5.677840979894002, val_loss: 5.825114285945892
Epoch 14/40, loss: 5.51163941025734, val_loss: 5.636732923984527
Epoch 15/40, loss: 5.390263547499974, val_loss: 5.689323616027832
Epoch 16/40, loss

In [30]:
# Évaluer le modèle
model.eval()
all_preds = []
all_preds_proba = []
all_labels = []

with torch.no_grad():
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(torch.concat((torch.tensor(input_ids, dtype=torch.float32), (torch.tensor(attention_mask, dtype=torch.float32))), 1))
        all_preds_proba.extend(outputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(np.argmax(labels.cpu(), axis=1))

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_sur_gpu.pth')

<ipython-input-30-011be65ae68d>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.concat((torch.tensor(input_ids, dtype=torch.float32), (torch.tensor(attention_mask, dtype=torch.float32))), 1))


Accuracy: 0.6521739130434783


In [31]:
# Évaluer le modèle
model.eval()
all_preds = []
all_preds_proba = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(torch.concat((torch.tensor(input_ids, dtype=torch.float32), (torch.tensor(attention_mask, dtype=torch.float32))), 1))
        all_preds_proba.extend(outputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(np.argmax(labels.cpu(), axis=1))

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_sur_gpu.pth')

<ipython-input-31-263c2d124ce8>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.concat((torch.tensor(input_ids, dtype=torch.float32), (torch.tensor(attention_mask, dtype=torch.float32))), 1))


Accuracy: 0.203125


## SentenceBERT

### Labels normaux

In [182]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from sentence_transformers import SentenceTransformer

# Charger votre dataframe
# Supposons que votre dataframe a deux colonnes : 'Text' pour les phrases et 'Label' pour les labels

# Convertir la colonne 'Label' en type 'category'
df['label'] = df['label'].astype('category')

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2)

# Charger le modèle Sentence-Transformers
sentencebert_model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Encoder les textes avec Sentence-Transformers
train_embeddings = sentencebert_model.encode(X_train.tolist(), convert_to_tensor=True)
test_embeddings  = sentencebert_model.encode(X_test.tolist(), convert_to_tensor=True)
valid_embeddings  = sentencebert_model.encode(X_val.tolist(), convert_to_tensor=True)

In [213]:
# Définir le modèle de classification
class ClassificationModel(torch.nn.Module):
    def __init__(self, num_labels):
        super(ClassificationModel, self).__init__()
        self.b = torch.nn.BatchNorm1d(768)
        self.fc = torch.nn.Linear(768, 1024)
        self.r = torch.nn.ReLU()
        self.s = torch.nn.Sigmoid()
        self.d = torch.nn.Dropout(p=0.05)
        self.fc1 = torch.nn.Linear(1024, 1024)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.fc3 = torch.nn.Linear(512, num_labels)

    def forward(self, x):
      x = self.b(x)
      x = self.d(self.r(self.fc(x)))
      x = self.d(self.r(self.fc1(x)))
      x = self.d(self.r(self.fc2(x)))
      x = self.s(self.d(self.fc3(x)))
      return x

# Initialiser le modèle de classification
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#def au desus
model = ClassificationModel(num_labels = 15).to('cuda')

In [214]:
# Définir le DataLoader
batch_size   = 32
train_loader = DataLoader(TensorDataset(train_embeddings.to(device), torch.tensor(np.array(pd.get_dummies(y_train)), dtype=torch.float32).to('cuda')), batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(TensorDataset(test_embeddings.to(device) , torch.tensor(np.array(pd.get_dummies(y_val)), dtype=torch.float32).to('cuda')), batch_size=batch_size, shuffle=False)
valid_loader  = DataLoader(TensorDataset(valid_embeddings.to(device) ,torch.tensor(np.array(pd.get_dummies(y_test)), dtype=torch.float32).to('cuda')), batch_size=batch_size, shuffle=False)

# Définir l'optimiseur
optimizer = AdamW(model.parameters())

num_epochs = 15

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_loss_valid = 0.0

    for batch in train_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    for batch in valid_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss_valid += loss.item()
    average_loss_valid = total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}, Val_loss: {average_loss_valid}')

# Évaluer le modèle
model.eval()
all_preds  = []
all_labels = []
all_outputs = []

with torch.no_grad():
    for batch in test_loader:
        embeddings, labels = batch
        outputs = model(embeddings)
        all_outputs.append(outputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(np.argmax(labels.cpu(), axis=1))

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_with_sentence_transformers.pth')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/15, Loss: 2.3778330115384834, Val_loss: 2.3541369438171387
Epoch 2/15, Loss: 2.3266551882721656, Val_loss: 2.3639424482981366
Epoch 3/15, Loss: 2.332838385604149, Val_loss: 2.3494441350301107
Epoch 4/15, Loss: 2.323184667631637, Val_loss: 2.3543374220530193
Epoch 5/15, Loss: 2.3254714788392534, Val_loss: 2.3561765511830646
Epoch 6/15, Loss: 2.3317595138106237, Val_loss: 2.3578744252522785
Epoch 7/15, Loss: 2.328877626463424, Val_loss: 2.4020838419596355
Epoch 8/15, Loss: 2.373049031856448, Val_loss: 2.4012661298116047
Epoch 9/15, Loss: 2.3724748700164087, Val_loss: 2.409251848856608
Epoch 10/15, Loss: 2.367571420447771, Val_loss: 2.4067376613616944
Epoch 11/15, Loss: 2.374218375183815, Val_loss: 2.4090646266937257
Epoch 12/15, Loss: 2.3730479340220607, Val_loss: 2.403959067662557
Epoch 13/15, Loss: 2.3804264844850054, Val_loss: 2.398223352432251
Epoch 14/15, Loss: 2.380105589711389, Val_loss: 2.410118405024211
Epoch 15/15, Loss: 2.3764875323273413, Val_loss: 2.3981983184814455


Meilleur résultat obtenue : 48%

In [189]:
np.round(np.mean(np.array(outputs.cpu().numpy()), axis = 0), 2)

array([0.01, 0.09, 1.  , 0.79, 0.49, 0.02, 0.15, 0.02, 0.03, 0.2 , 0.97,
       0.02, 0.03, 0.02, 0.01], dtype=float32)

### Education sub

In [144]:
len(np.where(df.label == 'Education')[0])/2

366.0

In [145]:
ind_E = rd.sample(list(np.where(df.label == 'Education')[0]), int(len(np.where(df.label == 'Education')[0])/2))
other = np.where(df.label != 'Education')[0]
total = ind_E + list(other)
print(len(total))
df = df.loc[total].reset_index()

1914


In [146]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from sentence_transformers import SentenceTransformer

# Charger votre dataframe
# Supposons que votre dataframe a deux colonnes : 'Text' pour les phrases et 'Label' pour les labels

# Convertir la colonne 'Label' en type 'category'
df['label'] = df['label'].astype('category')

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2)

# Charger le modèle Sentence-Transformers
sentencebert_model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Encoder les textes avec Sentence-Transformers
train_embeddings = sentencebert_model.encode(X_train.tolist(), convert_to_tensor=True)
test_embeddings  = sentencebert_model.encode(X_test.tolist(), convert_to_tensor=True)
valid_embeddings  = sentencebert_model.encode(X_val.tolist(), convert_to_tensor=True)


In [158]:
#def au desus
model = ClassificationModel(num_labels = 15).to('cuda')

In [148]:
import random as rd

In [159]:
# Définir le DataLoader
batch_size   = 32
train_loader = DataLoader(TensorDataset(train_embeddings.to(device), torch.tensor(np.array(pd.get_dummies(y_train)), dtype=torch.float32).to('cuda')), batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(TensorDataset(test_embeddings.to(device) , torch.tensor(np.array(pd.get_dummies(y_val)), dtype=torch.float32).to('cuda')), batch_size=batch_size, shuffle=False)
valid_loader  = DataLoader(TensorDataset(valid_embeddings.to(device) ,torch.tensor(np.array(pd.get_dummies(y_test)), dtype=torch.float32).to('cuda')), batch_size=batch_size, shuffle=False)

# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 30

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_loss_valid = 0.0

    for batch in train_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    for batch in valid_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss_valid += loss.item()
    average_loss_valid = total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}, Val_loss: {average_loss_valid}')

# Évaluer le modèle
model.eval()
all_preds  = []
all_labels = []
all_outputs = []

with torch.no_grad():
    for batch in test_loader:
        embeddings, labels = batch
        outputs = model(embeddings)
        all_outputs.append(outputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(np.argmax(labels.cpu(), axis=1))

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_with_sentence_transformers.pth')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/30, Loss: 2.6974444124433727, Val_loss: 2.684552768866221
Epoch 2/30, Loss: 2.668373419178857, Val_loss: 2.654196818669637
Epoch 3/30, Loss: 2.626983218722873, Val_loss: 2.6008724570274353
Epoch 4/30, Loss: 2.5735697084003024, Val_loss: 2.5611055294672647
Epoch 5/30, Loss: 2.534786641597748, Val_loss: 2.5175463954607644
Epoch 6/30, Loss: 2.4992118742730884, Val_loss: 2.5048141876856485
Epoch 7/30, Loss: 2.479844219154782, Val_loss: 2.502945144971212
Epoch 8/30, Loss: 2.4629050956832037, Val_loss: 2.4950044751167297
Epoch 9/30, Loss: 2.4698729581303067, Val_loss: 2.4822464187939963
Epoch 10/30, Loss: 2.456186811129252, Val_loss: 2.4736207326253257
Epoch 11/30, Loss: 2.448614093992445, Val_loss: 2.4827253222465515
Epoch 12/30, Loss: 2.4508000678486295, Val_loss: 2.4750072360038757
Epoch 13/30, Loss: 2.451392028066847, Val_loss: 2.4824753204981485
Epoch 14/30, Loss: 2.449343787299262, Val_loss: 2.479873021443685
Epoch 15/30, Loss: 2.4477699796358743, Val_loss: 2.479937573273977
Ep

In [160]:
np.round(np.mean(np.array(outputs.cpu().numpy()), axis = 0), 2)

array([0.01, 0.19, 0.99, 0.83, 0.42, 0.01, 0.11, 0.02, 0.21, 0.27, 0.99,
       0.1 , 0.19, 0.01, 0.03], dtype=float32)

In [161]:
pd.get_dummies(y_train).columns

CategoricalIndex(['Autos & Vehicles', 'Comedy', 'Education', 'Entertainment',
                  'Film & Animation', 'Gaming', 'Howto & Style', 'Music',
                  'News & Politics', 'Nonprofits & Activism', 'People & Blogs',
                  'Pets & Animals', 'Science & Technology', 'Sports',
                  'Travel & Events'],
                 categories=['Autos & Vehicles', 'Comedy', 'Education', 'Entertainment', ..., 'Pets & Animals', 'Science & Technology', 'Sports', 'Travel & Events'], ordered=False, dtype='category')

# Classic Machine Learning

## ML sur SentenceBERT

In [162]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pipeline

In [163]:
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(sentencebert_model.encode(X_train.tolist()),y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [88]:
tfsvmpred = clf.predict(sentencebert_model.encode(X_test.tolist()))
accuracy_score(y_test,tfsvmpred)

0.4934210526315789

## ML sur DistBERT

In [250]:
X_train, y_train = train_df['text'], train_df['label']
X_test, y_test = test_df['text'], test_df['label']

In [251]:
len(train_df['text']), len(train_df['label'])

(1909, 1909)

In [252]:
X_train = torch.cat([train_tokens['input_ids'], train_tokens['attention_mask']], 1)
X_test = torch.cat([test_tokens['input_ids'], test_tokens['attention_mask']], 1)
y_train, y_test = train_df['label'], test_df['label']

In [253]:
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(sentencebert_model.encode(X_train.tolist()),y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [254]:
tfsvmpred = clf.predict(sentencebert_model.encode(X_test.tolist()))
accuracy_score(y_test,tfsvmpred)

0.2484375

In [170]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [255]:
predovr = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,predovr)

0.1671875

In [256]:
clf = RandomForestClassifier(n_estimators = 500, max_depth=12, random_state=0)
clf.fit(X_train, y_train)
accuracy_score(clf.predict(X_test),y_test)

0.228125